In [1]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

# API 키가 제대로 로드되었는지 확인하는 코드 (선택 사항이지만 추천)
if not os.environ.get("GROQ_API_KEY"):
    raise ValueError("GROQ_API_KEY가 .env 파일에 없거나 로드되지 않았습니다.")

In [2]:
# === 1. 라이브러리 임포트 및 설정 ===
import os
# import json # 더 이상 JSON을 사용하지 않으므로 삭제하거나 주석 처리합니다.
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.prebuilt import ToolNode


# --- .env 파일 로드 ---
load_dotenv()
if not os.environ.get("GROQ_API_KEY"):
    raise ValueError("GROQ_API_KEY가 .env 파일에 없거나 로드되지 않았습니다.")

# === 2. 커스텀 파서로 카페 메뉴 데이터 로드 (수정된 부분) ===
def parse_menu_file(file_path: str) -> dict:
    """
    "메뉴명: 설명. 가격" 형식의 텍스트 파일을 읽어 딕셔너리로 변환합니다.
    """
    menu_db = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            # 빈 줄이나 콜론이 없는 줄은 건너뜁니다.
            if not line or ':' not in line:
                continue
            
            # 첫 번째 ':'를 기준으로 메뉴 이름과 나머지 부분을 분리합니다.
            menu_name, details = [part.strip() for part in line.split(':', 1)]
            
            # '₩' 기호를 기준으로 설명과 가격을 분리합니다.
            if '₩' in details:
                # rsplit을 사용해 오른쪽부터 나누어 설명에 '₩'가 있어도 대응 가능
                description, price = [part.strip() for part in details.rsplit('₩', 1)]
                price = '₩' + price # '₩' 기호를 다시 붙여줍니다.
            else:
                # 가격 정보가 없는 경우
                description = details
                price = "가격 정보 없음"

            menu_db[menu_name] = {"가격": price, "설명": description}
            
    return menu_db

try:
    # 올바른 상대 경로로 파일을 로드하고 커스텀 함수로 파싱합니다.
    cafe_menu_db = parse_menu_file('../data/cafe_menu_data.txt')
    if not cafe_menu_db:
        raise ValueError("메뉴 데이터를 파싱하지 못했습니다. 파일 내용이 비어있거나 형식이 다릅니다.")
    print("✅ 카페 메뉴 데이터를 커스텀 파서로 성공적으로 불러왔습니다.")
except FileNotFoundError:
    raise ValueError("`../data/cafe_menu_data.txt` 파일을 찾을 수 없습니다. 프로젝트 구조와 파일 경로를 확인해주세요.")


# === 3. 카페 메뉴 검색 도구 정의 ===
@tool
def search_cafe_menu(query: str) -> dict:
    """사용자가 질문한 카페 메뉴(음료 또는 디저트)에 대한 정보를 찾습니다."""
    print(f"--- 도구 실행: search_cafe_menu(query='{query}') ---")
    results = {}
    for menu, info in cafe_menu_db.items():
        if menu.lower() in query.lower():
            results[menu] = info
    if not results and query.lower() in "라떼":
        for menu, info in cafe_menu_db.items():
            if "라떼" in menu:
                results[menu] = info
    if not results:
        return {"메시지": f"'{query}'에 해당하는 메뉴를 찾을 수 없습니다."}
    return results

# === 4. Agent 상태 정의 ===
class AgentState(MessagesState):
    pass

# === 5. Agent, 도구, 그래프 노드 정의 ===
tools = [search_cafe_menu]
llm = ChatGroq(model_name="llama3-8b-8192")
system_prompt = (
    "당신은 친절한 카페 직원 Agent입니다. "
    "사용자의 질문에 답변하기 위해 `search_cafe_menu` 도구를 사용할 수 있습니다. "
    "사용자가 '라떼 종류' 와 같이 포괄적인 질문을 하면, '라떼' 라는 키워드로 도구를 검색하세요. "
    "도구를 사용한 후에는, 그 결과를 바탕으로 사용자에게 자연스러운 문장으로 답변을 정리해서 제공해야 합니다. "
    "불필요하게 도구를 반복해서 호출하지 마세요."
    "**매우 중요: 사용자가 한국어로 질문하고 있으므로, 당신의 모든 최종 답변은 반드시 한국어로만 작성해야 합니다. 절대로 영어나 다른 언어를 사용하지 마세요.**"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
agent_runnable = prompt | llm.bind_tools(tools)

def call_agent_node(state: AgentState):
    print("--- Agent 노드 실행 ---")
    response = agent_runnable.invoke(state)
    return {"messages": [response]}

tool_node = ToolNode(tools)

# === 6. 조건부 엣지 및 그래프 구조 ===
def should_continue(state: AgentState) -> str:
    if tool_calls := state["messages"][-1].tool_calls:
        return "tools"
    return "final_answer"

def format_final_response(state: AgentState):
    print("--- 최종 답변 생성 ---")
    final_answer: AIMessage = state["messages"][-1]
    clean_content = final_answer.content.replace('<tool-use>{"tool_calls":[]}</tool-use>', '').strip()
    if clean_content:
        print("🤖 카페 직원의 답변:\n\n" + clean_content)
    else:
        print("🤖 카페 직원: 더 이상 드릴 정보가 없습니다. 다른 것이 궁금하신가요?")
    return {"messages": [final_answer]}

# === 7. 그래프 생성 및 실행 ===
builder = StateGraph(AgentState)
builder.add_node("agent", call_agent_node)
builder.add_node("tools", tool_node)
builder.add_node("final_answer", format_final_response)
builder.set_entry_point("agent")
builder.add_conditional_edges("agent", should_continue)
builder.add_edge("tools", "agent")
builder.add_edge("final_answer", END)
graph = builder.compile()

def run_agent(question: str):
    print(f"\n===== 사용자 질문: {question} =====\n")
    inputs = {"messages": [HumanMessage(content=question)]}
    graph.invoke(inputs, {"recursion_limit": 10})

# === 8. 테스트 시나리오 실행 ===
try:
    run_agent("아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.")
    run_agent("라떼 종류에는 어떤 메뉴들이 있고 각각의 특징은 무엇인가요?")
    run_agent("디저트 메뉴 중에서 티라미수에 대해 자세히 설명해주세요.")
except Exception as e:
    print(f"\n오류가 발생했습니다: {e}")

✅ 카페 메뉴 데이터를 커스텀 파서로 성공적으로 불러왔습니다.

===== 사용자 질문: 아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요. =====

--- Agent 노드 실행 ---
--- 도구 실행: search_cafe_menu(query='아메리카노 과 아이스 아메리카노 차이점') ---
--- Agent 노드 실행 ---
--- 최종 답변 생성 ---
🤖 카페 직원의 답변:

아메리카노는 신선한 에스프레소에 물을 더한 클래식 커피입니다. 가격은 ₩4,000입니다. 

이제 아이스 아메리카노의 차이점을 알려드리겠습니다. 아이스 아메리카노는 일반 아메리카노와는 다르게 아이스 커피를 사용하여 무더운 날에 Refreshing하게 즐길 수 있습니다. 

따라서, 아이스 아메리카노는 일반 아메리카노와 가격이 같아 ₩4,000입니다.

===== 사용자 질문: 라떼 종류에는 어떤 메뉴들이 있고 각각의 특징은 무엇인가요? =====

--- Agent 노드 실행 ---
--- 도구 실행: search_cafe_menu(query='라떼') ---
--- Agent 노드 실행 ---
--- 최종 답변 생성 ---
🤖 카페 직원의 답변:

라떼 종류에는 카페 라떼, 바닐라 라떼, 초콜릿 라떼 등이 있습니다. 카페 라떼는 부드러운 우유와 에스프레소의 조화로 구성되어 있으며, 가격은 ₩4,500입니다. 바닐라 라떼는 달콤한 바닐라 시럽이 들어간 라떼로, 가격은 ₩5,000입니다. 초콜릿 라떼는 진한 초콜릿과 우유가 만난 음료로, 가격은 ₩5,500입니다.

===== 사용자 질문: 디저트 메뉴 중에서 티라미수에 대해 자세히 설명해주세요. =====

--- Agent 노드 실행 ---
--- 도구 실행: search_cafe_menu(query='티라미수') ---
--- Agent 노드 실행 ---
--- 최종 답변 생성 ---
🤖 카페 직원의 답변:

티라미수라는 디저트 메뉴는 카페의 메뉴에 없습니다. 다른 디저트 메뉴를 추천해 드리겠습니다. 카페에서는 